In [1]:
!pip install geopy

In [3]:
import sklearn

print(sklearn.__version__)

1.2.2


In [1]:
import pandas as pd
from geopy.distance import geodesic
import re

In [23]:
# 讀取事故資料作為df_a
df_a = pd.read_csv('事故熱點.csv')

# 讀取測站資料作為df_b
df_b = pd.read_csv('氣象測站清單20230726.csv')

# 讀取測速資料作為df_c
df_c = pd.read_csv('臺中市_camera.csv')

In [62]:
df_b["資料起始日期"][0]
	

'2002/1/1'

In [3]:
# 將測速資料的經緯度轉為數值
df_c['緯度'] = pd.to_numeric(df_c['緯度'], errors='coerce')
df_c['經度'] = pd.to_numeric(df_c['經度'], errors='coerce')

# 將經緯度為缺失值的那筆資料刪掉
df_c = df_c.dropna(subset=['緯度', '經度'])

In [3]:
# 日期的正規表達式
patternYears_month_date = r"(\d+)/(\d+)/(\d+)"

In [24]:
# 將日期的年月日分別切割之後放在list裡
def spilt_date(date):
    date_list = []
    match = re.search(patternYears_month_date, date)
    date_list.append(match.group(1))
    date_list.append(match.group(2))
    date_list.append(match.group(3))
    return date_list

# 該方法用來判斷該場車禍是否在指定測站的營運時間內，a、b、c值分別為六都車禍的日期、測站開始日期、測站撤銷日期
def during_activation_True_False(a, b, c):
    try:
        accident_date = spilt_date(a)
        station_start_date = spilt_date(b)
    
    except AttributeError:
        return_ture_false= False
        return return_ture_false
    
    if station_start_date[0]>accident_date[0]:
        after_start_date = False
        
    elif station_start_date[0] == accident_date[0]:
        
        if station_start_date[1]>accident_date[1]:
            after_start_date = False
            
        elif station_start_date[1] == accident_date[1]:
            
            if station_start_date[2] > accident_date[2]:
                after_start_date = False
                
            else:
                after_start_date = True
        else:
            after_start_date = True
                
    else:
        after_start_date = True
    
    if after_start_date:
        
        if pd.notna(c):
            station_stop_date = spilt_date(c)
            
            if accident_date[0] > station_stop_date[0]:
                return_ture_false = False
            
            elif accident_date[0] == station_stop_date[0]:
                
                if accident_date[1] > station_stop_date[1]:
                    return_ture_false = False
                    
                elif accident_date[1] == station_stop_date[1]:
                    
                    if accident_date[2] > station_stop_date[2]:
                        return_ture_false = False
                        
                    else: 
                        return_ture_false = True
                    
                else:
                    return_ture_false = True
                
            else: 
                return_ture_false = True
        
        else:
            return_ture_false= True
        
    else:
        return_ture_false= False
        
    return return_ture_false

# 對於df_a表中進來的每一筆資料，找到最接近的測站
def find_nearest_station():
    min_distance = float('inf')
    station_id = None
    for _, coord in df_b.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        
        if distance < min_distance:
            if during_activation_True_False(row['WHOLE_DATE'], coord['資料起始日期'], coord['撤站日期']):
                min_distance = distance
                print(min_distance)
                station_id = coord['站號']
            else:
                continue
    return station_id

# # 對於df_a表中進來的每一筆資料，找到最接近的測速，並限定範圍
def find_nearest_camera():
    min_distance = float('inf')
    camera_id = None
    limit_distance = 0.3
    for _, coord in df_c.iterrows():
        distance = geodesic((row['LATITUDE'], row['LONGITUDE']), (coord['緯度'], coord['經度'])).kilometers
        if distance < min_distance:
            min_distance = distance
            camera_id = coord['TEMP ID']
    if min_distance <= limit_distance:
        return camera_id

    else:
        return None

    

In [25]:
a = find_nearest_station()
a

17.08138095440058
6.176177198976939
4.524519370807105
4.170739887911762


'C0AD40'

In [66]:
# 建立空列
new_column = pd.Series

# 在指定位置插入空的列
df_a.insert(7, 'STATION_ID', new_column)

In [67]:
# 執行一次就好(如果不小心重複執行，就看下面的程式執行到哪筆再把0修改為看到的最後一筆的數字)
stop_point = 0

In [68]:
# 上面遇到未知狀況停止，除錯完之後再執行這個程式
for index, row in df_a.iterrows():
    if index >= stop_point:
        df_a.loc[index, 'STATION_ID'] = find_nearest_station()
        # df_a.loc[index, 'CAMERA_ID'] = find_nearest_camera()
        # if index % 1000 == 0:
        #     df_a.to_csv('台中市_A2_new_20230705.csv', index=False, encoding= 'UTF-8-sig')
        stop_point += 1
        print(f'第{index}筆已新增完畢')

12.86385041959388
11.40698728924955
4.241295832833296
2.805690365917714
1.9030975824017153
第0筆已新增完畢
11.00980422228789
2.6526945439458074
0.15735262445520176
第1筆已新增完畢
9.46717126602384
9.290853054161232
4.070119566861713
2.6947431503269605
2.1578808391502204
第2筆已新增完畢
10.83240002086834
6.548551054440445
2.017343813835742
第3筆已新增完畢
11.73019452144855
3.1683486210238754
0.766568460474879
第4筆已新增完畢
7.793499381650928
3.432144542343715
2.5865579382302837
第5筆已新增完畢
6.525937210756709
2.2586284613650007
0.7578055932720269
第6筆已新增完畢
17.51256597037733
14.109359199310576
7.570727565994515
0.8820506847721402
第7筆已新增完畢
13.436899578012401
4.450645449126024
2.004322583337269
第8筆已新增完畢
7.853337690821697
5.11705727611114
4.782477247814875
3.2809937850945308
2.2787178426870036
第9筆已新增完畢
5.281592181628874
3.821683827197418
1.8974399255857648
0.8453454335034588
第10筆已新增完畢
5.890341672102265
3.524332282045892
2.092617102017341
0.8937623320916024
第11筆已新增完畢
15.937382147607707
8.198348803548523
6.018261222603977
2.3859107

In [69]:
# 儲存結果到新的CSV檔案(檔名命名:縣市名稱+事故資料新增測站ID和測速ID+當下日期.csv)
df_a.to_csv('六都事故熱點_new_20230721.csv', index=False, encoding= 'UTF-8-sig')

In [70]:
df_a["STATION_ID"]

0      C0AI30
1      C0AI30
2      C0AJ80
3      C0ACA0
4      C0AI30
        ...  
225    C0V490
226    C0V660
227    C0V490
228    467441
229    C0V700
Name: STATION_ID, Length: 230, dtype: object